In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#IMPORTING THE BASIC LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
#matplotlib inline

In [ ]:
INPUT_FOLDER= "../input/"
print ('File Sizes:')
for f in os.listdir(INPUT_FOLDER):
    if 'zip' not in f:
       print (f.ljust(30) + str(round(os.path.getsize(INPUT_FOLDER +  f) / 1000, 2)) + ' KB')

In [ ]:
#CREATING A DATAFRAME FOR THE MAIN FILE TO BE USED IN THE CODE
main_file=pd.read_csv(INPUT_FOLDER + 'diabetes.csv')
main_file.shape

In [ ]:
main_file.describe()

In [ ]:
main_file.head()

In [ ]:
#COUNTING THE PEOPLE WITH AND WITHOUT DIABETES
main_file.groupby("Outcome").size()

In [ ]:
main_file.hist(figsize=(10,8))
plt.figure()
plt.show()

**Replacing '0' values of the columns mentioned below with their respective column mode.
**   
* BMI
* BLOOD PRESSURE
* GLUCOSE

In [ ]:
bmi_mode=main_file["BMI"].mode()

In [ ]:
bmi_mode

In [ ]:
main_file=main_file.replace({'BMI': {0: 32}}) 

In [ ]:
bp_mode=main_file["BloodPressure"].mode()
bp_mode

In [ ]:
main_file=main_file.replace({'BloodPressure': {0: 70}}) 

In [ ]:
glu_mode=main_file["Glucose"].mode()
glu_mode

In [ ]:
main_file=main_file.replace({'Glucose': {0: 99.5}}) 

In [ ]:
main_file.describe()

In [ ]:
main_file.hist(figsize=(10,8))
plt.figure()
plt.show()

In [ ]:
%matplotlib inline
main_file.plot(kind= 'box' , subplots=True, layout=(3,3), sharex=False, sharey=False, figsize=(10,8))

In [ ]:
import seaborn as sns
sns.set(style="ticks")
sns.pairplot(main_file, hue="Outcome")

###### Separating the data into Train & Test (80/20 split)

In [ ]:
X = main_file.ix[:,0:8]
Y = main_file["Outcome"]
from sklearn import model_selection
X_train, X_test, Y_train, Y_test= model_selection.train_test_split(X, Y, test_size=0.2)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
len(Y_train)

In [ ]:
len(Y_test)

###### Importing different models to check for the best accuracy 

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier


models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [ ]:
results = []
names = []
for name,model in models:
    kfold = model_selection.KFold(n_splits=10)
    cv_result = model_selection.cross_val_score(model,X_train,Y_train, cv = kfold,scoring = "accuracy")
    kfold = model_selection.KFold(n_splits=10)
    names.append(name)
    results.append(cv_result)
for i in range(len(names)):
    print(names[i],results[i].mean())

###### Visualizing the different model accuracies using a box plot

In [ ]:
ax = sns.boxplot(data=results)
ax.set_xticklabels(names)

In [ ]:
#FITTING THE LDA MODEL ON THE TEST DATASET
lda = LinearDiscriminantAnalysis()
lda.fit(X_train,Y_train)
predictions_lda = lda.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 

In [ ]:
print("Accuracy Score is:")
print(accuracy_score(Y_test, predictions_lda))
print()

In [ ]:
print("Classification Report:")
print(classification_report(Y_test, predictions_lda))

###### Creating a Confusion Matrix

In [ ]:
conf = confusion_matrix(Y_test,predictions_lda)
conf

In [ ]:
label = ["0","1"]
sns.heatmap(conf, annot=True, xticklabels=label, yticklabels=label)